In [ ]:
%md
### Kardiaflow - Silver Claims Enriched

**Source:** Snapshot join of Silver claims with current Silver providers

**Target:** `kardia_silver.silver_claims_enriched` (for Gold KPIs)

**Pattern:** LEFT JOIN from claims to SCD2 providers, keeping only `is_current = TRUE` row

**Trigger:** CTAS; rerun after any update to claims or providers

Notes: LEFT JOIN preserves all claims, even if provider record is missing.

In [ ]:
from kflow.notebook_utils import init

init()

In [0]:
%sql
-- 1. Ensure Silver DB exists
CREATE DATABASE IF NOT EXISTS kardia_silver;
USE kardia_silver;

In [ ]:
%sql
-- 2. Join claims to current provider attributes (no CTE – inline subquery)
CREATE OR REPLACE TABLE silver_claims_enriched AS
WITH current_providers AS (
  SELECT provider_id,
         provider_specialty,
         provider_location
  FROM   kardia_silver.silver_providers
  WHERE  is_current = TRUE
)
SELECT
    c.claim_id,
    c.patient_id,
    c.provider_id,
    c.claim_amount,
    c.claim_date,
    c.diagnosis_code,
    c.procedure_code,
    c.claim_status,
    c.claim_type,
    c.claim_submission_method,
    c._ingest_ts,
    c._batch_id,
    c._source_file,
    cp.provider_specialty,
    cp.provider_location
FROM kardia_silver.silver_claims c
LEFT JOIN current_providers cp
  ON c.provider_id = cp.provider_id;

In [0]:
%sql
-- 3. Preview: Top 5 enriched claims
SELECT *
FROM kardia_silver.silver_claims_enriched
ORDER BY claim_date DESC
LIMIT 5;